# Amazon Recommendation System Based On Preview

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [ ]:
import kagglehub
import os
import shutil
# Download latest version
path = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset")
data_dir = "./dataset"

os.makedirs(data_dir, exist_ok=True)
shutil.copytree(path, data_dir, dirs_exist_ok=True)

print("Dataset đã được lưu tại:", os.path.abspath(data_dir))

Dataset đã được lưu tại: /home/dikhangcshcmut/Code/code_for_fun/Project/RecommendationProduct/AmazonRecommendationBasedOnPreview/datasets


## Reading dataset

In [12]:
dataset = pd.read_csv(data_dir+"/amazon.csv")

print(f"Number of rows: {dataset.shape}")
print(f"Number of column: {dataset.columns.tolist()}")

Number of rows: (1465, 16)
Number of column: ['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'discount_percentage', 'rating', 'rating_count', 'about_product', 'user_id', 'user_name', 'review_id', 'review_title', 'review_content', 'img_link', 'product_link']


## Preprocessing dataset

In [19]:
# Checking number of missing value
print(dataset.isna().sum())

import pandas as pd
import numpy as np

def fill_missing_auto(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()
    
    for col in df_copy.columns:
        if pd.api.types.is_numeric_dtype(df_copy[col]):
            mean_val = df_copy[col].mean(skipna=True)
            df_copy[col] = df_copy[col].fillna(mean_val)
        else:
            df_copy[col] = df_copy[col].where(df_copy[col].notna(), np.nan)
    
    return df_copy
numeric_cols = ["discounted_price", "actual_price", "discount_percentage", "rating", "rating_count"]

for col in numeric_cols:
    dataset[col] = pd.to_numeric(dataset[col], errors="coerce")

# Sau đó fill missing auto
dataset = fill_missing_auto(dataset)
print(dataset.isna().sum())

dataset = fill_missing_auto(dataset)
print(dataset.isna().sum())

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64
product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64
product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0

In [20]:
print(dataset["discount_percentage"][:10])

0    0.64
1    0.43
2    0.90
3    0.53
4    0.61
5    0.85
6    0.65
7    0.23
8    0.50
9    0.33
Name: discount_percentage, dtype: float64


In [21]:
import re
def clear_special_character(x):
    if pd.isna(x):
        return np.nan
    
    x = str(x)
    x = re.sub(r"[^0-9.]", "", x)
    return x if x != "" else np.nan


def clear_price(x):
    x = clear_special_character(x)
    try:
        return float(x)
    except:
        return np.nan
    
def clear_percent(x):
    x = clear_special_character(x)
    try:
        return float(x) / 100.0
    except:
        return np.nan

In [22]:
# Aplly for columns needed
dataset["discounted_price"] = dataset["discounted_price"].apply(clear_price)
dataset["actual_price"] = dataset["actual_price"].apply(clear_price)
dataset["discount_percentage"] = dataset["discount_percentage"].apply(clear_percent)

print(dataset[["discounted_price", "actual_price", "discount_percentage"]].head(10))

   discounted_price  actual_price  discount_percentage
0            399.00        1099.0               0.0064
1            199.00         349.0               0.0043
2            199.00        1899.0               0.0090
3            329.00         699.0               0.0053
4            154.00         399.0               0.0061
5            149.00        1000.0               0.0085
6            176.63         499.0               0.0065
7            229.00         299.0               0.0023
8            499.00         999.0               0.0050
9            199.00         299.0               0.0033
